#### Case 2 - DDF Best Question Sequence

> Primary Goal: Optimize the order of the DDF questions to reduce diagnostic time and improve efficiency.

Input Data:

- **Questions File (PROD_NXT_question.csv)**: Contains question UUIDs, question content, duration, and expertise level.
- **Actions File (actions-9aba6d6a38c6423da035ea1be76a3cd9.csv)**: Details the sequence of executed actions for the DDF.
- **Solutions File (solutions-9aba6d6a38c6423da035ea1be76a3cd9.csv)**: Contains resolved failure modes (FMs).

### 1. Filter and Validate Data

- Confirm that only solved cases (Solved = YES) are selected from the filtered AP records.
    - Cross-check UUIDs:
        - DDF UUIDs link all files: Questions, Actions, and Solutions.
        - Ensure that the data reflects the same DDF version.
- Combine the Data:
    - Join questions.csv, actions.csv, and solutions.csv using DDF UUIDs.
    

In [2]:
import os
import pandas as pd

# Define file paths
questions_file = "PROD_NXT_question.csv"
actions_folder = "act-and-sol/actions/"
solutions_folder = "act-and-sol/solutions/"

# Load and preprocess questions data
questions = pd.read_csv(questions_file)[['DDF GUID', 'Question GUID', 'Duration']].drop_duplicates()
ddf_guid = '9aba6d6a38c6423da035ea1be76a3cd9'
questions = questions[questions['DDF GUID'] == ddf_guid]
questions['Order'] = range(1, len(questions) + 1)  # Assign question order
question_mapping = questions.set_index('Question GUID')['Order'].to_dict()

# Initialize list to store results
all_sequences = []

# Initialize a global sequence ID counter
global_sequence_id = 1

# Iterate through all action files
for action_file in os.listdir(actions_folder):
    if action_file.endswith(".csv"):
        action_path = os.path.join(actions_folder, action_file)
        
        # Find corresponding solution file
        solution_file = action_file.replace("actions", "solutions")
        solution_path = os.path.join(solutions_folder, solution_file)
        
        if not os.path.exists(solution_path):
            print(f"Solution file missing for {action_file}")
            continue
        
        # Load actions and solutions
        actions = pd.read_csv(action_path)
        solutions = pd.read_csv(solution_path)

        # Filter actions where SolvedIssue.text == 'YES'
        actions = actions[actions['SolvedIssue.text'].str.strip().str.upper() == 'YES']

        # Merge actions with questions
        merged_actions_questions = pd.merge(
            actions,
            questions,
            left_on='DdfAction.QuestionGlobalID.text',
            right_on='Question GUID',
            how='left'
        )
        merged_actions_questions['QuestionNumber'] = merged_actions_questions['DdfAction.QuestionGlobalID.text'].map(question_mapping)

        # Merge with solutions to include failure mode descriptions
        merged_data = pd.merge(
            merged_actions_questions,
            solutions,
            left_on='DdfFm1.text',
            right_on='DdfFm1.text',
            how='left'
        ).drop_duplicates()

        # Convert Answer (YES/NO) to descriptive text
#         merged_data['Answer_Text'] = merged_data['DdfAction.Answer.text'].str.strip().str.upper().map({'YES': 'Yes', 'NO': 'No'}).fillna('-')
        merged_data['Answer_Text'] = merged_data['DdfAction.Answer.text'].str.strip().str.upper().map({'YES': 'Yes', 'NO': 'No'})
        # Filter Applicable Failure Modes
        merged_data = merged_data[merged_data['DdfSolution.Applicable.text'].str.strip().str.upper() == 'YES']

        # Create Sequences for All Failure Modes
        sequence_table = []

        unique_failure_modes = merged_data['DdfSolution.Description.text'].dropna().unique()

        for failure_mode in unique_failure_modes:
            # Filter data for the current Failure Mode
            fm_data = merged_data[merged_data['DdfSolution.Description.text'] == failure_mode]

            # Initialize a row for the sequence table
            sequence_row = {'Sequence ID': global_sequence_id, 'Failure Mode': failure_mode}

            # Populate answers for each question and track sequence/duration
            sequence = []
            cumulative_duration = 0
            resolved = False  # Flag to stop processing after a resolving question is encountered

            for i in range(1, len(questions) + 1):
                if resolved:  # Stop processing once the issue is resolved
                    sequence_row[f'Q{i}'] = '-'
                    continue

                answer_data = fm_data[fm_data['QuestionNumber'] == i]
                if not answer_data.empty:
                    answer = answer_data['Answer_Text'].iloc[0]
                    duration = answer_data['Duration'].iloc[0]

                    sequence_row[f'Q{i}'] = answer
                    if answer in ['Yes', 'No']:
                        sequence.append(f'Q{i}')
                        cumulative_duration += duration

                    if answer == 'Yes':  # Resolve issue
                        resolved = True
                else:
                    sequence_row[f'Q{i}'] = '-'

            # Add cumulative duration and sequence order
            sequence_row['Total Duration'] = cumulative_duration
            sequence_row['Sequence'] = " > ".join(sequence) if sequence else '-'

            # Append the row to the sequence table
            sequence_table.append(sequence_row)

            # Increment the global sequence ID
            global_sequence_id += 1

        # Convert the sequence table to a DataFrame and add to all_sequences
        sequence_df = pd.DataFrame(sequence_table)
        all_sequences.append(sequence_df)

# Combine all sequences into one DataFrame
final_sequences = pd.concat(all_sequences, ignore_index=True)

# Save the combined sequences to a CSV for future use
final_sequences.to_csv("combined_sequences.csv", index=False)

# Output the final combined sequences
display(final_sequences)

,Sequence ID,Failure Mode,Q1,Q2,Q3,Q4,Q5,Q6,Total Duration,Sequence
0,1,LH need to be step initialized,No,Yes,-,-,-,-,25,Q1 > Q2
1,2,System hickup,No,No,No,Yes,-,-,60,Q1 > Q2 > Q3 > Q4
2,3,LH board problem,Yes,-,-,-,-,-,15,Q1
3,4,2DIB connection or IMCR communication NOK,No,No,Yes,-,-,-,40,Q1 > Q2 > Q3
4,5,Broken PSU,No,No,No,No,Yes,-,90,Q1 > Q2 > Q3 > Q4 > Q5
5,6,2DIB connection or IMCR communication NOK,No,No,Yes,-,-,-,40,Q1 > Q2 > Q3
6,7,LH need to be step initialized,No,Yes,-,-,-,-,25,Q1 > Q2
7,8,LH board problem,Yes,-,-,-,-,-,15,Q1
8,9,System hickup,No,No,No,Yes,-,-,60,Q1 > Q2 > Q3 > Q4
9,10,System hickup,No,No,No,Yes,-,-,60,Q1 > Q2 > Q3 > Q4


In [4]:
# Define the specific rows to remove
rows_to_remove = [
    {"Failure Mode": "Broken 2DIB box", "Sequence": "Q1 > Q3 > Q5"},
    {"Failure Mode": "Possible damaged cables between IMCR and Flexwave", "Sequence": "Q1 > Q3 > Q5"},
    {"Failure Mode": "System hickup", "Sequence": "Q1 > Q3 > Q5"},
]

# Iterate through the rows and remove them
for row in rows_to_remove:
    final_sequences = final_sequences[
        ~((final_sequences['Failure Mode'] == row["Failure Mode"]) &
          (final_sequences['Sequence'] == row["Sequence"]))
    ]

# Save the filtered DataFrame to a CSV
final_sequences.to_csv("filtered_sequences-v1.csv", index=False)

# Output the filtered sequences
display(final_sequences)

,Sequence ID,Failure Mode,Q1,Q2,Q3,Q4,Q5,Q6,Total Duration,Sequence
0,1,LH need to be step initialized,No,Yes,-,-,-,-,25,Q1 > Q2
1,2,System hickup,No,No,No,Yes,-,-,60,Q1 > Q2 > Q3 > Q4
2,3,LH board problem,Yes,-,-,-,-,-,15,Q1
3,4,2DIB connection or IMCR communication NOK,No,No,Yes,-,-,-,40,Q1 > Q2 > Q3
4,5,Broken PSU,No,No,No,No,Yes,-,90,Q1 > Q2 > Q3 > Q4 > Q5
5,6,2DIB connection or IMCR communication NOK,No,No,Yes,-,-,-,40,Q1 > Q2 > Q3
6,7,LH need to be step initialized,No,Yes,-,-,-,-,25,Q1 > Q2
7,8,LH board problem,Yes,-,-,-,-,-,15,Q1
8,9,System hickup,No,No,No,Yes,-,-,60,Q1 > Q2 > Q3 > Q4
9,10,System hickup,No,No,No,Yes,-,-,60,Q1 > Q2 > Q3 > Q4
